## Example Altimetry Observation Data
The simulation time of the .m21fm file in this example has been intentionally kept short. This adjustment allows for faster calculations within the notebook, making it easier to demonstrate the general usage of the package.

Please note that due to the shortened simulation period, the quality of the results is not representative of a full-scale analysis. This example is intended solely for demonstration purposes and should not be used for detailed assessments or decision-making.

##### Import

In [1]:
from pathlib import Path
from datetime import datetime

import optuna
from mike_autocal.autocal import AutoCal
from mike_autocal.dataio import ObservationData, SimObsPair, SimulationData
from mike_autocal.measurement_fun import ManningFile
from mike_autocal.mikesimulation import Launcher, RunTimeEvaluation
from mike_autocal.objective_fun import AMEANOuterMetric, RMSEInnerMetric

ROOT_DIR = Path().resolve()

##### Basic autocal input

- `study_name` - Unique identifier for this calibration study using satellite altimetry data.
- `simfile` - Base MIKE model file to be calibrated.
- `evaluation_time` - Defines from which time range is used to evaluate the output files. Can e.g. be usefull to ignore warm up.
- `n_trials` - Number of optimization iterations to perform.
- `sampler` - The optuna sampler that is use for the study.
- `direction` - Indicates that the optimization should "minimize" or "maximize" the error metrics. Can be a list if multiple objectives are used.
- `inner_metric` -  Applied to each simulation-observation pair independently. Can be a list if multiple objectives are used.
- `outer_metric` - Combines results from multiple simulation-observation pairs into a single objective value. Can be a list if multiple objectives are used.

In [2]:
study_name = "example_satellite_altimetry"
simfile = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/sns_base.m21fm"
evaluation_time = slice(50, None)
n_trials = 11
sampler = optuna.samplers.GPSampler(seed=0)
direction = ["minimize"]
inner_metric = [RMSEInnerMetric()]
outer_metric = [AMEANOuterMetric()]
use_gpu=False

##### Create measurement functions

Measurement functions define the parameters that will be calibrated in the MIKE model. Meaningful ranges can be defined it wished.

In [3]:
measurement_functions = [
    ManningFile(
        filename= ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/conditions/ManningM.dfsu",
        item_name="manning",
        low=0.001,
        high=81.101,
        step=0.01,
    )]

##### Create a Mapping of Simulation and Observation Data (Simulation-Observation Pairs)

In order for the autocal system to recognize the observation data as altimetry data, we need to specify `pair_type="track"` in the `SimObsPair`. This indicates that the data corresponds to a tracking type of observation.

Additionally, to properly interpret the `dfs0` files associated with track-type observations, we need to specify the following three items for each observation:
- x-coordinate: The X-coordinate of the observation location.
- y-coordinate: The Y-coordinate of the observation location.
- measurement item: The specific measurement recorded (e.g., altitude, water level).

Thus, in the `ObservationData`, we set `item=[0,1,2]` to refer to the indices of the above-mentioned items in the `dfs0` file. This ensures that the system correctly interprets the coordinates and measurement data from the file.


In [4]:
base_sim_path = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/simulation/sns_base.m21fm - Result Files"
base_obs_path = ROOT_DIR / "mike_autocal/tests/data/simulation_data_sns/observations"

# Verify the simulation file exists
dfsu_file = base_sim_path / "Area.dfsu"

simobs = [
    SimObsPair(
        name="3a",
        pair_type="track",
        sim=SimulationData(
            file_path=dfsu_file,
            item=0,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "Altimetry_3a.dfs0",
            item=[0,1,2],  
        ),
    ),
    SimObsPair(
        name="3b",
        pair_type="track",
        sim=SimulationData(
            file_path=dfsu_file,
            item=0,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "Altimetry_3b.dfs0",
            item=[0,1,2],  
        ),
    ),
    SimObsPair(
        name="6a",
        pair_type="track",
        sim=SimulationData(
            file_path=dfsu_file,
            item=0,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "Altimetry_6a.dfs0",
            item=[0,1,2],  
        ),
    ),
    SimObsPair(
        name="j3",
        pair_type="track",
        sim=SimulationData(
            file_path=dfsu_file,
            item=0,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "Altimetry_j3.dfs0",
            item=[0,1,2],  
        ),
    ),
    SimObsPair(
        name="sa",
        pair_type="track",
        sim=SimulationData(
            file_path=dfsu_file,
            item=0,  
        ),
        obs=ObservationData(
            file_path = base_obs_path / "Altimetry_sa.dfs0",
            item=[0,1,2],  
        ),
    ),]

##### Create Launcher

In [5]:
rte = RunTimeEvaluation(
    simobs=simobs,
    inner_metric=inner_metric,
    outer_metric=outer_metric,
    frequency=50,
    logdir=Path(f"logs/simulation_{datetime.now().strftime('%Y%m%d_%H%M%S')}"),
)

launcher = Launcher(
    simfile=simfile,
    use_gpu=use_gpu,
    runtimeevaluation=rte,
)

##### Create autocal object

In [6]:
calibration = AutoCal(
    study_name=study_name,
    launcher=launcher,
    simobs=simobs,
    inner_metric=inner_metric,  
    outer_metric=outer_metric,
    n_trials=n_trials,
    direction=direction,
    sampler=sampler, 
    measurement_functions=measurement_functions,
    evaluation_time=evaluation_time,
    verbose=False,
    load_if_exists=False,
)
calibration.run()

[I 2025-04-21 15:00:23,709] A new study created in Journal with name: example_satellite_altimetry
INFO:autocal:------------------------------------ Trial: 0 ------------------------------------


Processing: 100%|██████████| 500/500 [00:43<00:00, 11.48step/s]
INFO:autocal:InnerEvaluation(metric=RMSE):
        values    n
name               
3a    0.129837  129
3b    0.331027  152
6a    0.249326  240
j3    0.245915  248
sa    0.089781  123
[I 2025-04-21 15:01:11,862] Trial 0 finished with value: 0.20917726330931416 and parameters: {'Manning_0': 44.510999999999996, 'Manning_1': 58.001, 'Manning_2': 48.891, 'Manning_3': 44.190999999999995, 'Manning_4': 34.361, 'Manning_5': 52.381, 'Manning_6': 35.491}. Best is trial 0 with value: 0.20917726330931416.
INFO:autocal:------------------------------------ Trial: 1 ------------------------------------
Processing: 100%|██████████| 500/500 [00:44<00:00, 11.17step/s]
INFO:autocal:InnerEvaluation(metric=RMSE):
        values    n
name               
3a    0.109094  129
3b    0.324532  152
6a    0.207971  240
j3    0.205144  248
sa    0.114870  123
[I 2025-04-21 15:02:00,399] Trial 1 finished with value: 0.19232227208177138 and parameters: {'

##### For testing
Trial 10 should have an objective value of `0.18750156325443176`